In [64]:
SYNA_PARENT_DIR = "/home/shd-sun-lab/SynapseNavigator"

In [65]:
import sys
import os
sys.path.append(SYNA_PARENT_DIR) # append the path of protgps
from argparse import Namespace
import pickle
import copy
import yaml
import requests
from tqdm import tqdm
from p_tqdm import p_map
import numpy as np
import pandas as pd
from collections import defaultdict
import torch 
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz
import protpy
from protpy import amino_acids as protpyAA
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
from protgps.utils.loading import get_object

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Functions

### Make sure you run these, but can be ignored after

In [67]:
#Compartment labels & helper

COMPARTMENTS = [
    "cytosol",
    "ER",
    "mitochondrion",
    "nucleus",
    "Excitatory Synapse",
    "Inhibitory Synapses",
]

# In the original code this is identical, but conceptually OLDCOMPS is
# "the order used in the committed code / old model".
OLDCOMPS = [
    "cytosol",
    "ER",
    "mitochondrion",
    "nucleus",
    "Excitatory Synapse",
    "Inhibitory Synapses",
]

def transform_y(y: torch.Tensor):
    """
    Convert labels from COMPARTMENTS order to OLDCOMPS order.
    """
    indices = torch.nonzero(y).flatten()
    new_indices = torch.tensor([OLDCOMPS.index(COMPARTMENTS[i]) for i in indices])
    return torch.zeros(len(OLDCOMPS)).scatter_(0, new_indices, 1)


In [68]:

UNIPROT_FASTA_URL = "https://rest.uniprot.org/uniprotkb/{}.fasta"
UNIPROT_JSON_URL  = "https://rest.uniprot.org/uniprotkb/{}.json"


def get_organism(uniprot_id: str) -> str:
    """
    Return the scientific name of the organism for a UniProt ID,
    or empty string if not found.
    """
    r = requests.get(UNIPROT_JSON_URL.format(uniprot_id))
    if r.status_code != 200:
        return ""
    js = r.json()
    if "organism" in js:
        return js["organism"].get("scientificName", "")
    return ""


def parse_fasta(f: str) -> str:
    """
    Parse fasta text and return the amino acid sequence.
    """
    seq = []
    for line in f.splitlines():
        if line.startswith(">"):
            continue
        seq.append(line.strip())
    return "".join(seq)

from typing import Optional

def get_protein_fasta(uniprot_id: str) -> Optional[str]:
    """
    Download protein sequence for a UniProt ID.
    Returns None if request fails.
    """
    r = requests.get(UNIPROT_FASTA_URL.format(uniprot_id))
    if r.status_code != 200:
        return None
    return parse_fasta(r.text)


In [ ]:
#Quick UniProt test
test_id = "O14983"
seq = get_protein_fasta(test_id)
org = get_organism(test_id)
print("ID:", test_id)
print("Organism:", org)
print("Sequence length:", len(seq) if seq else None)
print("First 60 AA:", (seq[:60] + "...") if seq else "None")



ID: O14983
Organism: Homo sapiens
Sequence length: 1001
First 60 AA: MEAAHAKTTEECLAYFGVSETTGLTPDQVKRNLEKYGLNELPAEEGKTLWELVIEQFEDL...


In [69]:
# [6] Model helper functions

def load_model(snargs):
    """
    Loads classifier model from args file + checkpoint path in snargs.model_path
    """
    modelpath = snargs.model_path
    model = get_object(snargs.lightning_name, "lightning")(snargs)
    model = model.load_from_checkpoint(
        checkpoint_path=modelpath,
        strict=not snargs.relax_checkpoint_matching,
        **{"args": snargs},
    )
    return model, snargs


def predict_condensates(model, sequences, batch_size=8, round_scores=True):
    """
    Run the model on a list of sequences and return scores tensor [N, n_labels].
    """
    all_scores = []
    model.eval()
    for i in tqdm(range(0, len(sequences), batch_size), ncols=100):
        batch = sequences[i : i + batch_size]
        with torch.no_grad():
            out = model.model({"x": batch})
        s = torch.sigmoid(out["logit"]).to("cpu")
        all_scores.append(s)
    scores = torch.vstack(all_scores)
    if round_scores:
        scores = torch.round(scores, decimals=3)
    return scores


def get_valid_rows(df, cols):
    """
    Keep only rows where all sequences in `cols` are shorter than 1800 AA.
    """
    rows_with_valid_seq_len = []
    for i in range(len(df)):
        if all(len(df.iloc[i][c]) < 1800 for c in cols):
            rows_with_valid_seq_len.append(i)
    return rows_with_valid_seq_len


In [70]:
#Load args + checkpoint from your SynapseNavigator folder

import glob

checkpoint_dir = os.path.join(
    SYNA_PARENT_DIR,
    "checkpoints",
    "protgps",
    "2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)",
)

print("Checkpoint dir:", checkpoint_dir)
print("Contents:", os.listdir(checkpoint_dir))

args_files = glob.glob(os.path.join(checkpoint_dir, "*.args"))
ckpt_files = glob.glob(os.path.join(checkpoint_dir, "*.ckpt"))

print("Found args files:", args_files)
print("Found ckpt files:", ckpt_files)

assert len(args_files) == 1, "Expected exactly one .args file here."
assert len(ckpt_files) == 1, "Expected exactly one .ckpt file here."

args_path = args_files[0]
ckpt_path = ckpt_files[0]

print("Using ARGS:", args_path)
print("Using CKPT:", ckpt_path)

# Load args
with open(args_path, "rb") as f:
    args = Namespace(**pickle.load(f))

# Fix up paths inside args
args.pretrained_hub_dir = os.path.join(SYNA_PARENT_DIR, "checkpoints", "esm2")
args.model_path         = ckpt_path
args.dataset_file_path  = os.path.join(SYNA_PARENT_DIR, "data", "dataset.json")

# Load model
model, args = load_model(args)
model = model.to(device)
model.eval()

print("Loaded model:", type(model))
print("Dataset name:", args.dataset_name)


Checkpoint dir: /home/shd-sun-lab/SynapseNavigator/checkpoints/protgps/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)
Contents: ['5f71fcd178bd6ceebc393003a6db70d9.args', '2025.10.17_Genes_Hein_2025_&_Marc_2023(1Synapse).xlsx:Zone.Identifier', '5f71fcd178bd6ceebc393003a6db70d9epoch=20.ckpt', '2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse).json', '2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)_with_preds.csv', '2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse).xlsx:Zone.Identifier', '2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse).xlsx', 'Recorded Data']
Found args files: ['/home/shd-sun-lab/SynapseNavigator/checkpoints/protgps/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)/5f71fcd178bd6ceebc393003a6db70d9.args']
Found ckpt files: ['/home/shd-sun-lab/SynapseNavigator/checkpoints/protgps/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)/5f71fcd178bd6ceebc393003a6db70d9epoch=20.ckpt']
Using ARGS: /home/shd-sun-lab/SynapseNavigator/checkpoints/protgps/2025.10.17_Genes_Hein_2025_&_Mar

Using cache found in /home/shd-sun-lab/SynapseNavigator/checkpoints/esm2/facebookresearch_esm_main
Using cache found in /home/shd-sun-lab/SynapseNavigator/checkpoints/esm2/facebookresearch_esm_main


Using ESM hidden layers 6
Loaded model: <class 'protgps.lightning.base.Base'>
Dataset name: protein_condensates_combined


# Predictions on Additional Data 

## Predict files can also be used

In [ ]:
#Load train/dev/test datasets from dataset
# Ignore, this is just to get an overview of original dataset size

train_dataset = get_object(args.dataset_name, "dataset")(args, "train")
dev_dataset   = get_object(args.dataset_name, "dataset")(args, "dev")
test_dataset  = get_object(args.dataset_name, "dataset")(args, "test")

print("Train/dev/test sizes:",
      len(train_dataset.dataset),
      len(dev_dataset.dataset),
      len(test_dataset.dataset))

train_sequences = set(d["x"] for d in (train_dataset.dataset + dev_dataset.dataset))
len(train_sequences)


100%|██████████| 6627/6627 [00:00<00:00, 149735.24it/s]


TRAIN DATASET CREATED FOR PROTEIN_CONDENSATES_COMBINED.
* 3502 Proteins.
* 1254 CYTOSOL -- 444 ER -- 519 MITOCHONDRION -- 854 NUCLEUS -- 260 EXCITATORY SYNAPSE -- 358 INHIBITORY SYNAPSES


100%|██████████| 6627/6627 [00:00<00:00, 285434.92it/s]


DEV DATASET CREATED FOR PROTEIN_CONDENSATES_COMBINED.
* 739 Proteins.
* 225 CYTOSOL -- 110 ER -- 104 MITOCHONDRION -- 199 NUCLEUS -- 59 EXCITATORY SYNAPSE -- 72 INHIBITORY SYNAPSES


100%|██████████| 6627/6627 [00:00<00:00, 270551.53it/s]

TEST DATASET CREATED FOR PROTEIN_CONDENSATES_COMBINED.
* 737 Proteins.
* 262 CYTOSOL -- 103 ER -- 95 MITOCHONDRION -- 170 NUCLEUS -- 71 EXCITATORY SYNAPSE -- 69 INHIBITORY SYNAPSES
Train/dev/test sizes: 3502 739 737


4200

In [71]:
#Load the Excel file you want to make predictions on

import pandas as pd

excel_path = "/home/shd-sun-lab/SynapseNavigator/checkpoints/protgps/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse).xlsx"

data = pd.read_excel(excel_path)
print("Loaded Excel:", excel_path)
print("Shape:", data.shape)

Loaded Excel: /home/shd-sun-lab/SynapseNavigator/checkpoints/protgps/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse).xlsx
Shape: (7011, 23)


In [72]:
# [10] Detect sequence or UniProt columns

import numpy as np

# Common sequence column names
possible_seq_cols = ["Sequence", "AA_sequence", "ProteinSequence", "WT_Sequence"]

# Common UniProt ID column names
possible_uniprot_cols = ["Entry", "UniProt", "Accession", "ProteinID"]

seq_col = next((c for c in possible_seq_cols if c in data.columns), None)
uniprot_col = next((c for c in possible_uniprot_cols if c in data.columns), None)

print("Detected sequence column:", seq_col)
print("Detected UniProt ID column:", uniprot_col)


Detected sequence column: Sequence
Detected UniProt ID column: Entry


In [38]:
# [11] Extract sequences depending on what the Excel provides
#If sequences are not present in the Excel, we fetch them from UniProt using the IDs provided

if seq_col is not None:
    # Use sequences directly
    print("Using sequences from Excel column:", seq_col)
    
    sequences = []
    valid_indices = []
    
    for idx, seq in data[seq_col].items():
        if isinstance(seq, str) and 0 < len(seq) < 1800:
            sequences.append(seq)
            valid_indices.append(idx)
    
    print("Number of valid sequences found:", len(sequences))

elif uniprot_col is not None:
    # Fallback: fetch sequences from UniProt
    print("Fetching sequences via UniProt column:", uniprot_col)
    protein_ids = data[uniprot_col].astype(str).tolist()
    
    from p_tqdm import p_map
    seqs = p_map(get_protein_fasta, protein_ids)
    
    sequences = []
    valid_indices = []
    
    for idx, seq in enumerate(seqs):
        if seq is not None and len(seq) < 1800:
            sequences.append(seq)
            valid_indices.append(idx)
    
    print("Valid sequences obtained from UniProt:", len(sequences))

else:
    raise ValueError("ERROR: Neither sequence nor UniProt ID column found.")


Using sequences from Excel column: Sequence
Number of valid sequences found: 6694


In [39]:
# [12] Run predictions on the sequences

print(f"Running predictions on {len(sequences)} sequences...")
scores = predict_condensates(model, sequences, batch_size=8, round_scores=True)

print("Score tensor shape:", scores.shape)


Running predictions on 6694 sequences...


100%|█████████████████████████████████████████████████████████████| 837/837 [03:07<00:00,  4.46it/s]

Score tensor shape: torch.Size([6694, 6])


In [40]:
# [13] Insert model scores into the DataFrame

import numpy as np

for j, comp in enumerate(OLDCOMPS):
    col_name = f"{comp.upper()}_Score"
    score_col = [np.nan] * len(data)
    
    for df_idx, score_vec in zip(valid_indices, scores):
        score_col[df_idx] = float(score_vec[j])
    
    data[col_name] = score_col

data.head()


,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Sequence,centrosome,cytosol,...,translation,Excitatory Synapse,Inhibitory Synapses,Dopaminergic Synapses,CYTOSOL_Score,ER_Score,MITOCHONDRION_Score,NUCLEUS_Score,EXCITATORY SYNAPSE_Score,INHIBITORY SYNAPSES_Score
0,A0A023T6R1,A0A023T6R1,unreviewed,A0A023T6R1_HUMAN,"Mago nashi protein (Mago-nashi homolog, isofor...",FLJ10292 hCG_1773848,Homo sapiens (Human),MAVASDFYLRYYVGHKGKFGHEFLEFEFRPDGKLRYANNSNYKNDV...,0,0,...,0,0,0,0,0.685,0.027,0.023,0.032,0.172,0.263
1,A0A024QYR6,A0A024QYR6,unreviewed,A0A024QYR6_HUMAN,"Phosphatidylinositol 3,4,5-trisphosphate 3-pho...",PTEN,Homo sapiens (Human),MERGGEAAAAAAAAAAAPGRGSESPVTISRAGNAGELVSPLLLPPT...,0,1,...,0,0,0,0,0.357,0.000,0.000,0.718,0.042,0.021
2,A0A024QYS2,A0A024QYS2,unreviewed,A0A024QYS2_HUMAN,Transmembrane 9 superfamily member,NaN,Homo sapiens (Human),MRPLPGALGVAAAAALWLLLLLLPRTRADEHEHTYQDKEEVVLWMN...,0,0,...,0,0,0,0,0.001,0.453,0.066,0.003,0.064,0.322
3,A0A024QYX0,A0A024QYX0,unreviewed,A0A024QYX0_HUMAN,Emopamil binding protein,EBP,Homo sapiens (Human),MTTNAGPLHPYWPQHLRLDNFVPNDRPTWHILAGLFSVTGVLVVTT...,0,0,...,0,0,0,0,0.043,0.894,0.007,0.030,0.020,0.026
4,A0A024QZ64,A0A024QZ64,unreviewed,A0A024QZ64_HUMAN,Fructose-bisphosphate aldolase (EC 4.1.2.13),NaN,Homo sapiens (Human),MPHSYPALSAEQKKELSDIALRIVAPGKGILAADESVGSMAKRLSQ...,0,1,...,0,0,0,0,0.689,0.002,0.007,0.055,0.121,0.182


In [41]:
# [14] Save output

out_path = excel_path.replace(".xlsx", "_with_preds.csv")
data.to_csv(out_path, index=False)

print("Saved predictions to:\n", out_path)


Saved predictions to:
 /home/shd-sun-lab/SynapseNavigator/checkpoints/protgps/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)/2025.10.17_Genes_Hein_2025_&_Marc_2023(2Synapse)_with_preds.csv


## Predictions on additional datasets (template)

This section is a template for running the trained SynapseNavigator model
on *any* new Excel dataset.

To use:
1. Point `excel_path` to your `.xlsx`.
2. Either:
   - set `seq_col` to the column containing amino acid sequences, **or**
   - set `uniprot_col` to the column containing UniProt IDs (if no sequences).
3. Run all cells below to produce a `<filename>_with_preds.csv` file.


In [ ]:
# [T1] CONFIG: point to your new Excel here

import os
import pandas as pd
import numpy as np

# Example: copy+adapt this line when someone adds a new dataset
excel_path = "/path/to/your/new_dataset.xlsx"  # <--- CHANGE THIS

data = pd.read_excel(excel_path)
print("Loaded Excel:", excel_path)
print("Shape:", data.shape)
data.head()


In [ ]:
# [T2] Detect / set sequence or UniProt ID column

# Option A: let the code auto-detect common names
possible_seq_cols     = ["Sequence", "AA_sequence", "ProteinSequence", "WT_Sequence"]
possible_uniprot_cols = ["Entry", "UniProt", "Accession", "ProteinID"]

seq_col = next((c for c in possible_seq_cols if c in data.columns), None)
uniprot_col = next((c for c in possible_uniprot_cols if c in data.columns), None)

print("Auto-detected sequence column:", seq_col)
print("Auto-detected UniProt column:", uniprot_col)

In [ ]:
# [T3] Build list of sequences to score

from p_tqdm import p_map

sequences = []
valid_indices = []

if seq_col is not None:
    print(f"Using sequences from column: {seq_col}")
    for idx, seq in data[seq_col].items():
        if isinstance(seq, str) and 0 < len(seq) < 1800:
            sequences.append(seq)
            valid_indices.append(idx)

elif uniprot_col is not None:
    print(f"Using UniProt IDs from column: {uniprot_col}")
    ids = data[uniprot_col].astype(str).tolist()
    seqs = p_map(get_protein_fasta, ids)
    for idx, seq in enumerate(seqs):
        if seq is not None and len(seq) < 1800:
            sequences.append(seq)
            valid_indices.append(idx)

else:
    raise ValueError("No usable sequence or UniProt column found.")

print("Number of valid sequences:", len(sequences))


In [ ]:
# [T4] Predict scores

scores = predict_condensates(model, sequences, batch_size=8, round_scores=True)
print("Scores shape:", scores.shape)


In [ ]:
# [T5] Add one score column per compartment

for j, comp in enumerate(OLDCOMPS):
    col_name = f"{comp.upper()}_Score"
    col_vals = [np.nan] * len(data)
    for df_idx, score_vec in zip(valid_indices, scores):
        col_vals[df_idx] = float(score_vec[j])
    data[col_name] = col_vals

data.head()


In [ ]:
# [T6] Save <original>_with_preds.csv next to input

out_path = excel_path.replace(".xlsx", "_with_preds.csv").replace(".xls", "_with_preds.csv")
data.to_csv(out_path, index=False)
print("Saved predictions to:", out_path)


# AUCs

In [ ]:
# === AUC EVALUATION (Self-contained) ===
# Uses the same `args` and the same loaded `model` already in memory.

from sklearn.metrics import roc_auc_score

print("Using dataset for AUCs:", args.dataset_file_path)

# 1) Load test dataset
test_dataset = get_object(args.dataset_name, "dataset")(args, "test")
print("Loaded test samples:", len(test_dataset.dataset))

# 2) Build test sequences and labels
test_x = [sample["x"] for sample in test_dataset.dataset]
test_y = [transform_y(sample["y"]) for sample in test_dataset.dataset]
test_y = torch.vstack(test_y)

# 3) Run model predictions (probabilities)
test_preds = predict_condensates(model, test_x, batch_size=32, round_scores=False)

print("Shapes -> y:", test_y.shape, " preds:", test_preds.shape)
print("\n=== ROC-AUC Scores ===")

# 4) Compute AUC per condensate
for j, comp in enumerate(OLDCOMPS):
    y_true = test_y[:, j].numpy()
    y_score = test_preds[:, j].numpy()
    try:
        auc = roc_auc_score(y_true, y_score)
        print(f"{comp:20s}  {auc:.3f}")
    except ValueError as e:
        print(f"{comp:20s}  AUC undefined ({e})")


# Analysis

In [73]:
# [AN1-TEMPLATE] Define sequences for attribution analysis
#
# HOW TO USE:
# - Fill the `sequences_for_ig` list with your own sequences.
# - Each entry is a dict with:
#     - "name":         an ID for the construct (for plots / tables)
#     - "condensate":   any label / class name you want to attach (optional, for you)
#     - "seed":         an integer identifier (optional, can be anything)
#     - "sequence":     the amino acid sequence (without tags like mCherry)
#
# You can reuse this template and change only `sequences_for_ig`.

sequences_for_ig = [
    {
        "condensate": "Synapse",    # e.g. "nucleolus", "synapse", "test_construct"
        "name": "VATG2",       # short ID
        "seed": 1,                     # any integer (or remove if not needed)
        "sequence": "MASQSQGIQQLLQAEKRAAEKVADARKRKARRLKQAKEEAQMEVEQYRREREHEFQSKQQAAMGSQGNLSAEVEQATRRQVQGMQSSQQRNRERVLAQLLGMVCDVRPQVHPNYRISA"  # <-- your AA seq
    },
    {
        "condensate": "Synapse",
        "name": "NeuM",
        "seed": 2,
        "sequence": "MLCCMRRTKQVEKNDDDQKIEQDGIKPEDKAHKAATKIQASFRGHITRKKLKGEKKDDVQAAEAEANKKDEAPVADGVEKKGEGTTTAEAAPATGSKPDEPGKAGETPSEEKKGEGDAATEQAAPQAPASSEEKAGSAETESATKASTDNSPSSKAEDAPAKEEPKQADVPAAVTAAAATTPAAEDAAAKATAQPPTETGESSQAEENIEAVDETKPKESARQDEGKEEEPEADQEHA"                 # another AA seq
    },
    # Add more dicts as needed
]

# Optional: define a tag that is always fused, or set to "" if not needed
mcherry = (
    ""
)
# If you don't want to fuse anything, just do:
# mcherry = ""

# For the rest of the IG pipeline, we will refer to this list:
experimental_sequences = sequences_for_ig
print(f"Prepared {len(experimental_sequences)} sequences for IG.")



Prepared 2 sequences for IG.


In [74]:
# [AN2] Visualization helper (Captum)

def visualize_text(datarecords, legend: bool = True) -> "HTML":
    dom = ["<table width: 100%>"]
    rows = [
        "<th>Amino Acid Importance</th>"
        "<th>Sample ID</th>"
        "<th>Target (score)</th>"
    ]
    for datarecord in datarecords:
        rows.append(
            "".join(
                [
                    "<tr>",
                    viz.format_word_importances(
                        datarecord.raw_input_ids, datarecord.word_attributions
                    ),
                    # just use true_class as-is (no split)
                    viz.format_classname(str(datarecord.true_class)),
                    viz.format_classname(
                        "{0} ({1:.2f})".format(
                            datarecord.pred_class, datarecord.pred_prob
                        )
                    ),
                    "<tr>",
                ]
            )
        )

    if legend:
        dom.append(
            '<div style="border-top: 1px solid; margin-top: 5px; \
            padding-top: 5px; display: inline-block">'
        )
        dom.append("<b>Legend: </b>")

        for value, label in zip([-1, 0, 1], ["Negative", "Neutral", "Positive"]):
            dom.append(
                '<span style="display: inline-block; width: 10px; height: 10px; \
                border: 1px solid; background-color: \
                {value}"></span> {label}  '.format(
                    value=viz._get_color(value), label=label
                )
            )
        dom.append("</div>")

    dom.append("".join(rows))
    dom.append("</table>")
    html = viz.HTML("".join(dom))
    viz.display(html)

    return html


In [75]:
# [AN3] Forward function for IG and Captum setup

model = model.to("cpu")
model.eval()

def forward(batch_tokens):
    """
    Forward pass that:
    - gets ESM representations
    - mean-pools across positions
    - passes through the MLP head
    - returns sigmoid probabilities
    """
    model.zero_grad()
    result = model.model.encoder.model(
        batch_tokens,
        repr_layers=[model.model.encoder.repr_layer],
        return_contacts=False,
    )
    hidden = result["representations"][model.model.encoder.repr_layer].mean(axis=1)
    scores = torch.sigmoid(model.model.mlp({"x": hidden})["logit"])
    return scores

alphabet = model.model.encoder.alphabet

lig = LayerIntegratedGradients(forward, model.model.encoder.model.embed_tokens)


In [76]:
# [AN4] Integrated Gradients for the experimental sequences

records = []
sequence_dict_copy = copy.deepcopy(experimental_sequences)

for sequence_dict in sequence_dict_copy:
    seq = sequence_dict["sequence"]
    input_seq = seq + mcherry

    # Baseline: CLS + [MASK] * len(seq) + EOS
    baseline = torch.tensor(
        [alphabet.cls_idx] + [alphabet.mask_idx] * len(input_seq) + [alphabet.eos_idx]
    ).unsqueeze(0)

    # Inputs: use ESM batch_converter
    fair_x = [(0, input_seq)]
    _, _, batch_tokens = model.model.encoder.batch_converter(fair_x)

    # Prediction for the full fusion
    with torch.no_grad():
        model.eval()
        out = model.model({"x": [input_seq]})
    probs = torch.sigmoid(out["logit"]).detach().cpu()
    pred_idx = probs.argmax().item()
    pred_class_name = OLDCOMPS[pred_idx]

    # Store model's predicted class info alongside original label
    sequence_dict["model_pred_class_idx"] = int(pred_idx)
    sequence_dict["model_pred_class_name"] = pred_class_name
    sequence_dict["model_pred_probs"] = probs.squeeze().tolist()

    # IG attributions w.r.t. the predicted class
    attributions, delta = lig.attribute(
        inputs=batch_tokens,
        baselines=baseline,
        return_convergence_delta=True,
        target=pred_idx,
        n_steps=50,
    )
    A = attributions.sum(-1)[0, 1:-1]  # drop CLS/EOS
    A = A / torch.norm(A)
    sequence_dict["attributions"] = A.tolist()

    # Build Captum visualization record
    record = viz.VisualizationDataRecord(
        word_attributions=A * 10,
        pred_prob=probs.max().item(),
        pred_class=pred_class_name,              # model's predicted class
        true_class=sequence_dict["name"],        # your sequence label / ID
        attr_class="-",
        attr_score=attributions[0, 1:-1].sum(),
        raw_input_ids=input_seq,
        convergence_score=delta,
    )
    records.append(record)

# Take a quick look at what model predicted vs your labels
sequence_dict_copy


[{'condensate': 'Synapse',
  'name': 'VATG2',
  'seed': 1,
  'sequence': 'MASQSQGIQQLLQAEKRAAEKVADARKRKARRLKQAKEEAQMEVEQYRREREHEFQSKQQAAMGSQGNLSAEVEQATRRQVQGMQSSQQRNRERVLAQLLGMVCDVRPQVHPNYRISA',
  'model_pred_class_idx': 4,
  'model_pred_class_name': 'Excitatory Synapse',
  'model_pred_probs': [0.06139502674341202,
   0.0011658864095807076,
   1.1990982784482185e-05,
   0.13033036887645721,
   0.9232924580574036,
   0.47822701930999756],
  'attributions': [0.04107086880367465,
   -0.27569252430774005,
   0.6031264164626648,
   -0.017165788675634522,
   0.0016837581281229576,
   0.007321488045300473,
   0.031743149265642916,
   -0.05979586770967675,
   0.055249870115039186,
   0.03399575650794573,
   0.09451724376733123,
   -0.08618136275329669,
   0.0426475515773979,
   0.017179287172215604,
   0.11569484279894426,
   0.06945644800351981,
   -0.06515974882636677,
   0.028947182604885296,
   0.035146172980187025,
   0.036831958599218784,
   0.051163747350563345,
   0.01829941594464258,


In [77]:
# [AN5] Visualize attributions and save to disk

html = visualize_text(records)

with open("html_file.html", "w") as f:
    f.write(html.data)

# Also save the attribution data as CSV
for sequence_dict in sequence_dict_copy:
    seq = sequence_dict["sequence"]
    sequence_dict["full_sequence"] = seq + mcherry

attr_df = pd.DataFrame(sequence_dict_copy)
attr_df.to_csv("attributions.csv", index=False)

attr_df.head()


VATG2,Excitatory Synapse (0.92)
NeuM,Excitatory Synapse (0.88)


,condensate,name,seed,sequence,model_pred_class_idx,model_pred_class_name,model_pred_probs,attributions,full_sequence
0,Synapse,VATG2,1,MASQSQGIQQLLQAEKRAAEKVADARKRKARRLKQAKEEAQMEVEQ...,4,Excitatory Synapse,"[0.06139502674341202, 0.0011658864095807076, 1...","[0.04107086880367465, -0.27569252430774005, 0....",MASQSQGIQQLLQAEKRAAEKVADARKRKARRLKQAKEEAQMEVEQ...
1,Synapse,NeuM,2,MLCCMRRTKQVEKNDDDQKIEQDGIKPEDKAHKAATKIQASFRGHI...,4,Excitatory Synapse,"[0.115046925842762, 0.001337190275080502, 1.78...","[0.07333355394692458, 0.17847705944933295, -0....",MLCCMRRTKQVEKNDDDQKIEQDGIKPEDKAHKAATKIQASFRGHI...
